In [1]:
import glob
import keras
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from keras.models import Model, Sequential
from keras.layers import Dense, Input, Conv2D, Conv2DTranspose, BatchNormalization, Reshape, LeakyReLU, Activation

Using TensorFlow backend.


In [2]:
image_ids = glob.glob('celeb/*')[:10000]

In [3]:
def get_generator():

    inp = Input((512,))
        
    x = Dense(8192)(inp)
    x = Reshape((4,4,512))(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
        
    x = Conv2DTranspose(256, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)        
    x = LeakyReLU(0.2)(x)        
    
    x = Conv2DTranspose(128, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)        
    x = LeakyReLU(0.2)(x)
    
    x = Conv2DTranspose(64, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)        
    x = LeakyReLU(0.2)(x)   
        
    x = Conv2DTranspose(3, kernel_size=5, strides=2, padding='same')(x)
    out = Activation('tanh')(x)
        
    model = Model(inp, out)    
        
    return model

In [4]:
def get_discriminator():    
    
    inp = Input((64,64,3))

    x = Conv2D(28, kernel_size=5, strides=2, padding='same')(inp)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(64, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)

    x = Conv2D(128, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    
    x = Conv2D(256, kernel_size=5, strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(0.2)(x)
    
    x = Reshape((4096,))(x)
    x = Dense(1)(x)
    out = Activation('sigmoid')(x)
    
    model = Model(inp, out)    
        
    return model

In [5]:
def get_images(batch=64):

    crop = (30, 55, 150, 175)
    batch_ids = np.random.choice(image_ids, batch)
    images = [np.array((Image.open(i).crop(crop)).resize((64,64))) for i in batch_ids]
    images = np.array(images)
    images = images/255
    images = images-0.5
    
    return np.array(images)

In [6]:
def train(batch, epochs):
    
    generator = get_generator()
    discriminator = get_discriminator()

    adam_generator = keras.optimizers.Adam(lr=0.0002, beta_1=0.5)
    adam_discriminator = keras.optimizers.Adam(lr=0.0002, beta_1=0.5)

    discriminator.compile(loss='binary_crossentropy', optimizer=adam_discriminator, metrics=['accuracy'])
    generator.compile(loss='binary_crossentropy', optimizer=adam_generator, metrics=['accuracy'])  
    
    noise = Input((512,))
    z = discriminator(generator(noise))
    discriminator.trainable = False
    combined = Model(noise, z)
    combined.compile(loss='binary_crossentropy', optimizer=adam_generator, metrics=['accuracy'])
    discriminator.trainable = True

    print_every = 50
    no_batch = 1000//batch

    for e in range(epochs): 
    
        for i in range(no_batch):
            
            noise = np.random.uniform(-1, 1, size=(batch, 512))
            real_images = get_images(batch)
            fake_images = generator.predict(noise)
            
            images = np.concatenate((real_images, fake_images))
            labels = np.concatenate((np.ones((batch,1)), np.zeros((batch,1))))
        
            discriminator.train_on_batch(images, labels)
            discriminator.trainable = False    
            combined.train_on_batch(noise, np.ones((batch,1)))
            discriminator.trainable = True
            
            if i%print_every == 0:
                print("epochs: %d/%d batchs: %d/%d" %((e+1),epochs,(i+1),no_batch))

In [7]:
train(50, 1)
a = generator.predict(np.random.uniform(-1, 1, size=(1, 512)))
plt.imshow((a[0]+0.5)*255)

epochs: 1/1 batchs: 1/20


NameError: name 'generator' is not defined